<a href="https://colab.research.google.com/github/amiranissian/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [15]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [16]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [20]:
def get_fashion_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [21]:
batch_size = 128
epochs = 15

In [22]:
# Get the training data
x_train, y_train = get_fashion_mnist_data()[0]

4422102/4422102 [==============================] - 0s 0us/step


In [23]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 50s 117ms/step - loss: 0.6101 - accuracy: 0.7845 - val_loss: 0.4580 - val_accuracy: 0.8335
Epoch 2/15
422/422 [==============================] - 52s 123ms/step - loss: 0.4032 - accuracy: 0.8560 - val_loss: 0.3900 - val_accuracy: 0.8597
Epoch 3/15
422/422 [==============================] - 49s 116ms/step - loss: 0.3601 - accuracy: 0.8711 - val_loss: 0.3658 - val_accuracy: 0.8687
Epoch 4/15
422/422 [==============================] - 48s 113ms/step - loss: 0.3323 - accuracy: 0.8811 - val_loss: 0.3480 - val_accuracy: 0.8755
Epoch 5/15
422/422 [==============================] - 48s 113ms/step - loss: 0.3138 - accuracy: 0.8869 - val_loss: 0.3256 - val_accuracy: 0.8832
Epoch 6/15
422/422 [==============================] - 48s 113ms/step - loss: 0.2991 - accuracy: 0.8920 - val_loss: 0.3094 - val_accuracy: 0.8895
Epoch 7/15
422/422 [==============================] - 48s 113ms/step - loss: 0.2831 - accuracy: 0.8986 - val_loss: 0.3328 - val_ac

In [24]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 57.5 MB/s 
     |████████████████████████████████| 168 kB 45.9 MB/s 
     |████████████████████████████████| 62 kB 751 kB/s 
     |████████████████████████████████| 168 kB 49.5 MB/s 
     |████████████████████████████████| 166 kB 54.8 MB/s 
     |████████████████████████████████| 166 kB 54.5 MB/s 
     |████████████████████████████████| 162 kB 46.2 MB/s 
     |████████████████████████████████| 162 kB 40.4 MB/s 
     |████████████████████████████████| 158 kB 43.9 MB/s 
     |████████████████████████████████| 157 kB 53.5 MB/s 
     |████████████████████████████████| 157 kB 62.4 MB/s 
     |████████████████████████████████| 157 kB 62.6 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |████████████████████████████

In [25]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [26]:
batch_size = 128
epochs = 15
wandb.init(project="övning3_v2",
           config={"batch_size": batch_size, "epochs": epochs})

wandb: Currently logged in as: amir_anissian. Use `wandb login --relogin` to force relogin


In [27]:
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
422/422 [==============================] - ETA: 0s - loss: 0.2041 - accuracy: 0.9261

wandb: Adding directory to artifact (/content/wandb/run-20221213_084143-12bdfpzy/files/model-best)... Done. 0.0s


422/422 [==============================] - 58s 138ms/step - loss: 0.2041 - accuracy: 0.9261 - val_loss: 0.2702 - val_accuracy: 0.9055
Epoch 2/15
422/422 [==============================] - ETA: 0s - loss: 0.1956 - accuracy: 0.9300

wandb: Adding directory to artifact (/content/wandb/run-20221213_084143-12bdfpzy/files/model-best)... Done. 0.0s


422/422 [==============================] - 50s 118ms/step - loss: 0.1956 - accuracy: 0.9300 - val_loss: 0.2675 - val_accuracy: 0.9063
Epoch 3/15
422/422 [==============================] - ETA: 0s - loss: 0.1893 - accuracy: 0.9310

wandb: Adding directory to artifact (/content/wandb/run-20221213_084143-12bdfpzy/files/model-best)... Done. 0.0s


422/422 [==============================] - 51s 120ms/step - loss: 0.1893 - accuracy: 0.9310 - val_loss: 0.2636 - val_accuracy: 0.9067
Epoch 4/15
422/422 [==============================] - 49s 116ms/step - loss: 0.1858 - accuracy: 0.9329 - val_loss: 0.2789 - val_accuracy: 0.9002
Epoch 5/15
422/422 [==============================] - 48s 114ms/step - loss: 0.1797 - accuracy: 0.9358 - val_loss: 0.2728 - val_accuracy: 0.9032
Epoch 6/15
422/422 [==============================] - 48s 114ms/step - loss: 0.1719 - accuracy: 0.9377 - val_loss: 0.2782 - val_accuracy: 0.9047
Epoch 7/15
422/422 [==============================] - 52s 124ms/step - loss: 0.1675 - accuracy: 0.9395 - val_loss: 0.2714 - val_accuracy: 0.9085
Epoch 8/15
422/422 [==============================] - 48s 115ms/step - loss: 0.1625 - accuracy: 0.9421 - val_loss: 0.2704 - val_accuracy: 0.9088
Epoch 9/15
422/422 [==============================] - 48s 114ms/step - loss: 0.1576 - accuracy: 0.9434 - val_loss: 0.2756 - val_accuracy: 0.9

accuracy,▁▂▂▃▄▄▅▅▆▆▆▇▇▇█
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▇▇▆▆▅▄▄▃▃▃▂▂▂▁
val_accuracy,▅▆▆▁▃▅▇▇▇▁▃█▇▆█
val_loss,▂▂▁▃▂▃▂▂▃█▃▂▄▃▂
accuracy,0.95189
best_epoch,2
best_val_loss,0.26355
epoch,14
loss,0.1325
val_accuracy,0.90967
